In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
gender = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
gender.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [6]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of women who survived:", rate_men)

% of women who survived: 0.18890814558058924


In [7]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"] 
#타깃변수인 "Survived" 열을 선택하여 y에 저장

features = ["Pclass", "Sex", "SibSp", "Parch"] 
#모델에서 사용할 특성 변수들을 지정
X = pd.get_dummies(train_data[features])
#특성 변수들을 one-hot 인코딩하여 더미 변수로 변환 
#이는 범주형 데이터를 모델에 사용하기 적합한 형태로 변환하는 과정
X_test = pd.get_dummies(test_data[features])

**특성 변수를 one-hot 인코딩하여 더미 변수로 변환**하는 것은 범주형 데이터를 모델이 이해할 수 있는 형태로 변환하는 과정. 
일반적으로 기계 학습 모델은 숫자 데이터를 입력으로 받기 때문에, 범주형 변수를 숫자로 변환해주어야 합니다. One-hot 인코딩은 이를 수행하는 일반적인 방법 중 하나입니다.

예를 들어, "Sex" 특성이 'male'과 'female' 두 개의 범주를 가진 경우, one-hot 인코딩을 통해 새로운 더미 변수를 생성합니다. **각각의 더미 변수는 해당 범주에 속하는지 여부를 0 또는 1로 나타냅니다. 이렇게 하면 모델이 성별에 대한 정보를 숫자로 인식할 수 있게 됩니다.**

예시:

"Sex" 특성이 'male' 또는 'female'일 경우,
"Sex_male": 1 (male인 경우)
"Sex_female": 0 (male인 경우)
또는
"Sex_male": 0 (female인 경우)
"Sex_female": 1 (female인 경우)
이러한 방식으로 one-hot 인코딩을 적용하면 **각 범주형 변수가 별도의 더미 변수로 분리되어, 모델이 범주 정보를 이해하고 학습할 수 있게 됩니다.** 
**pandas의 get_dummies 함수**를 사용하면 이러한 변환을 쉽게 수행할 수 있습니다.

In [8]:
model = RandomForestClassifier(n_estimators = 100, max_depth=5, random_state=1)
model.fit(X,y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


RandomForestClassifier 모델을 생성합니다. 이 모델은 100개의 결정 트리를 사용하며, 각 트리의 최대 깊이는 5로 제한되어 있습니다. random_state는 랜덤 시드로, 재현성을 위해 설정된 값입니다.

model.fit(X, y): 훈련 데이터 X와 타겟 변수 y를 사용하여 모델을 학습합니다.

predictions = model.predict(X_test): 테스트 데이터 X_test를 사용하여 생존 여부를 예측합니다. 이 결과는 predictions 변수에 저장됩니다.

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions}): 예측 결과를 새로운 데이터프레임으로 만듭니다. 이 데이터프레임은 'PassengerId'와 모델의 예측 결과 'Survived'를 포함합니다.

output.to_csv('submission.csv', index=False): CSV 파일로 결과를 저장합니다. 파일명은 'submission.csv'이며, 인덱스는 저장하지 않도록 설정되었습니다.

print("Your submission was successfully saved!"): 파일 저장이 성공적으로 완료되었음을 알리는 메시지를 출력합니다.